### ⚠️ این نوت‌بوک برای بهبود کیفیت ویدیوهای با تایم پایین تا 60 ثانیه مناسب است.
### ⏳ پردازش ویدیوهای با تایم بیشتر از این، زمان‌بر هست.


In [ ]:

#@title نصب و فعال‌سازی محیط

from IPython.display import display, HTML, clear_output

# ۱. شروع فرآیند نصب
print("در حال نصب کتابخانه‌ها و آماده‌سازی محیط... لطفاً صبور باشید.")

# ۲. اجرای دستورات نصب در پس‌زمینه
!rm -rf /content/GFPGAN
%cd /content/
!git clone https://github.com/TencentARC/GFPGAN.git &> /dev/null
%cd /content/GFPGAN
!pip install -r requirements.txt &> /dev/null
!python setup.py develop &> /dev/null
!pip install realesrgan &> /dev/null
!pip install gradio &> /dev/null

# ۳. پاکسازی خروجی نصب و نمایش پیام نهایی
clear_output()

# ۴. ساخت و نمایش محتوای HTML
html_content = """
<div style="border: 2px solid #4CAF50; padding: 15px; border-radius: 8px; text-align: center;">
  <p style="font-size: 16px;"><b>✅ نصب با موفقیت انجام شد.</b></p>
  <p>👇 برای جلوگیری از قطع شدن اتصال، دکمه پخش زیر را بزنید و سراغ مراحل بعد بروید.</p>
  <audio src="https://raw.githubusercontent.com/KoboldAI/KoboldAI-Client/main/colab/silence.m4a" controls>
</div>
"""

display(HTML(html_content))

In [ ]:
#@title دانلود مدل‌ها

import requests
import sys
import shutil
import os
import urllib.request
from tqdm import tqdm
from IPython.display import clear_output

base_path = "/content"

# --- فیکس فایل degradations برای BasicSR ---
def fix_degradations():
    url = "https://raw.githubusercontent.com/NeuralFalconYT/GFPGAN-video-upscale/main/degradations.py"
    filename = "/content/degradations.py"
    full_version = sys.version.split(' ')[0]
    major_minor_version = '.'.join(full_version.split('.')[:2])
    basicsr_path = f"/usr/local/lib/python{major_minor_version}/dist-packages/basicsr/data/degradations.py"

    # Send a GET request to the URL
    response = requests.get(url)
    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Write the content to a file
        with open(filename, 'wb') as file:
            file.write(response.content)
        try:
            shutil.copy("/content/degradations.py", basicsr_path)
            print(f"✅ فایل degradations.py در مسیر {basicsr_path} کپی شد")
        except Exception as e:
            print(f"⚠️ خطا در کپی کردن degradations.py: {e}")
            print(f"لطفاً متغیر 'basicsr_path' را بروزرسانی کنید - '{major_minor_version}' را با نسخه فعلی Python در Google Colab جایگزین کنید")
    else:
        print("❌ دانلود فایل degradations.py شکست خورد")

# اجرای فیکس
fix_degradations()

# --- توابع کمکی ---
def conditional_download(url, download_file_path, display_name=None):
    if os.path.exists(download_file_path):
        print(f"فایل {os.path.basename(download_file_path)} از قبل وجود دارد. دانلود نادیده گرفته شد.")
        return

    file_display_name = display_name or os.path.basename(download_file_path)
    print(f"در حال دانلود {file_display_name}...")
    base_dir = os.path.dirname(download_file_path)
    os.makedirs(base_dir, exist_ok=True)

    try:
        request = urllib.request.urlopen(url)
        total = int(request.headers.get('Content-Length', 0))
        with tqdm(total=total, desc=f'دانلود {file_display_name}', unit='B', unit_scale=True, unit_divisor=1024) as progress:
            urllib.request.urlretrieve(url, download_file_path, reporthook=lambda count, block_size, total_size: progress.update(block_size))
        print(f"✅ دانلود {file_display_name} با موفقیت انجام شد.")
    except Exception as e:
        print(f"❌ خطا در دانلود فایل: {e}")

# --- تعریف لینک‌ها و مسیرها ---
script_url = 'https://raw.githubusercontent.com/menchbaz/GFPGAN/main/inference_gfpgan.py'
script_path = f"{base_path}/GFPGAN/inference_gfpgan.py"

gfpgan_model_url = 'https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.4.pth'
gfpgan_model_path = f"{base_path}/GFPGAN/experiments/pretrained_models/GFPGANv1.3.pth"

realesrgan_model_url = 'https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.1/RealESRGAN_x2plus.pth'
realesrgan_model_path = f"{base_path}/GFPGAN/gfpgan/weights/RealESRGAN_x2plus.pth"

detection_model_url = 'https://github.com/xinntao/facexlib/releases/download/v0.1.0/detection_Resnet50_Final.pth'
detection_model_path = f"{base_path}/GFPGAN/gfpgan/weights/detection_Resnet50_Final.pth"

parsing_model_url = 'https://github.com/xinntao/facexlib/releases/download/v0.2.2/parsing_parsenet.pth'
parsing_model_path = f"{base_path}/GFPGAN/gfpgan/weights/parsing_parsenet.pth"

# --- شروع دانلود ---
print("در حال دانلود اسکریپت اصلی از مخزن شما...")
os.system(f"wget -q -O {script_path} {script_url}")

conditional_download(gfpgan_model_url, gfpgan_model_path, display_name="GFPGAN Model")
conditional_download(realesrgan_model_url, realesrgan_model_path, display_name="RealESRGAN Model")
conditional_download(detection_model_url, detection_model_path, display_name="Detection Model")
conditional_download(parsing_model_url, parsing_model_path, display_name="Parsing Model")

clear_output()
print("✅ تمام مدل‌ها، اسکریپت اصلی و فیکس degradations.py با موفقیت آماده شدند.")
print("💡 حالا می‌توانید با scale factor 4 عملیات را انجام دهید.")

In [ ]:


#@title ⚙️ آماده‌سازی

base_path="/content"
import cv2
import shutil
import os
import subprocess
from IPython.display import clear_output, display, Video
from google.colab import files
from tqdm import tqdm
import time

# اطمینان از اینکه در دایرکتوری درست هستیم
if 'GFPGAN' not in os.getcwd():
    os.chdir(f"{base_path}/GFPGAN")

def get_video_details(video_path):
    try:
        ffprobe_fps_cmd = f"ffprobe -v error -select_streams v:0 -show_entries stream=r_frame_rate -of default=noprint_wrappers=1:nokey=1 '{video_path}'"
        fps_str = subprocess.check_output(ffprobe_fps_cmd, shell=True, text=True).strip()
        num, den = map(int, fps_str.split('/'))
        fps = num / den if den != 0 else 25
        ffprobe_audio_cmd = f"ffprobe -i '{video_path}' -show_streams -select_streams a -loglevel error"
        audio_output = subprocess.check_output(ffprobe_audio_cmd, shell=True, text=True).strip()
        has_audio = bool(audio_output)
        print(f"✅ مشخصات ویدیو تشخیص داده شد: نرخ فریم: {fps:.2f}, دارای صدا: {has_audio}")
        return fps, has_audio
    except Exception as e:
        print(f"⚠️ اخطار: تشخیص خودکار نرخ فریم شکست خورد. از مقدار پیش‌فرض 25 FPS استفاده می‌شود.")
        return 25, True

def make_video_with_correct_fps(original_fps):
    restoredFramesPath = f'{base_path}/GFPGAN/results/restored_imgs/'
    output_path = f'{base_path}/video_only.mp4'
    dir_list = sorted([f for f in os.listdir(restoredFramesPath) if f.endswith('.jpg')])
    if not dir_list: return None

    print(f"در حال بازسازی ویدیو از {len(dir_list)} فریم...")
    sample_img = cv2.imread(os.path.join(restoredFramesPath, dir_list[0]))
    height, width, layers = sample_img.shape
    size = (width, height)
    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), original_fps, size)

    with tqdm(total=len(dir_list), desc="بازسازی ویدیو", unit="فریم") as pbar:
        for filename in dir_list:
            img = cv2.imread(os.path.join(restoredFramesPath, filename))
            if img is not None:
                out.write(img)
                pbar.update(1)

    out.release()
    print("✅ بازسازی ویدیو با موفقیت انجام شد.")
    return output_path

def generate_random_name(video_filename):
    import uuid
    base_name, extension = os.path.splitext(os.path.basename(video_filename))
    random_uuid = str(uuid.uuid4().hex)[:6]
    return f"{base_path}/{base_name}_GFPGAN_{random_uuid}{extension}"

def monitor_gfpgan_progress():
    """نمایش پیشرفت پردازش GFPGAN"""
    results_dir = f"{base_path}/GFPGAN/results/restored_imgs/"
    if not os.path.exists(results_dir):
        return

    processed_files = len([f for f in os.listdir(results_dir) if f.endswith('.jpg')])
    return processed_files

def run_gfpgan_and_create_final_video(video_path):
    original_fps, has_audio = get_video_details(video_path)
    images_dir = f"{base_path}/images"
    if os.path.exists(images_dir): shutil.rmtree(images_dir)
    os.makedirs(images_dir)

    print("در حال استخراج فریم‌ها...")
    extract_cmd = f"ffmpeg -i '{video_path}' '{images_dir}/%06d.jpg' -hide_banner -loglevel error"
    if os.system(extract_cmd) != 0:
        print("❌ استخراج فریم‌ها شکست خورد."); return None

    # شمارش فریم‌های استخراج شده
    frame_count = len([f for f in os.listdir(images_dir) if f.endswith('.jpg')])
    print(f"✅ {frame_count} فریم استخراج شد.")

    if os.path.exists(f'{base_path}/GFPGAN/results'): shutil.rmtree(f'{base_path}/GFPGAN/results')

    print("\n⚙️ تنظیمات بهینه‌سازی GPU:")
    print("- bg_tile: 800 (افزایش استفاده از GPU)")
    print("- نمایش پیشرفت در حین اجرا")

    print("🚀 شروع فرآیند بهبود کیفیت با GPU بهینه...")
    gfpgan_cmd = f"python inference_gfpgan.py -i {images_dir} -o results -v 1.3 -s 4 --bg_upsampler realesrgan --bg_tile 800"

    # اجرای پردازش در پس‌زمینه با نمایش پیشرفت
    import subprocess
    import threading
    import time

    def show_progress():
        """نمایش پیشرفت در حین اجرا"""
        while True:
            if os.path.exists(f'{base_path}/GFPGAN/results/restored_imgs/'):
                processed = monitor_gfpgan_progress()
                if processed > 0:
                    progress_percent = (processed / frame_count) * 100
                    print(f"\r⏳ پیشرفت: {processed}/{frame_count} فریم ({progress_percent:.1f}%)", end="", flush=True)
                    if processed >= frame_count:
                        print()  # خط جدید
                        break
            time.sleep(2)

    # شروع thread برای نمایش پیشرفت
    progress_thread = threading.Thread(target=show_progress, daemon=True)
    progress_thread.start()

    start_time = time.time()
    result = os.system(gfpgan_cmd)
    end_time = time.time()

    if result != 0:
        print(f"❌ فرآیند بهبود کیفیت شکست خورد. کد خطا: {result}")
        return None

    processing_time = end_time - start_time
    print(f"✅ فرآیند بهبود کیفیت در {processing_time:.1f} ثانیه تکمیل شد")
    print(f"⚡ سرعت پردازش: {frame_count/processing_time:.1f} فریم در ثانیه")

    video_only_path = make_video_with_correct_fps(original_fps)
    if not video_only_path:
        print("❌ بازسازی ویدیو شکست خورد."); return None

    final_video_path = generate_random_name(video_path)
    print("در حال ترکیب نهایی ویدیو با صدای اصلی...")
    if has_audio:
        merge_cmd = (f"ffmpeg -i '{video_only_path}' -i '{video_path}' -map 0:v:0 -map 1:a:0 "
                     f"-c:v libx264 -profile:v baseline -pix_fmt yuv420p -c:a aac -b:a 192k -shortest "
                     f"'{final_video_path}' -y -hide_banner -loglevel error")
    else:
        merge_cmd = f"ffmpeg -i '{video_only_path}' -c:v libx264 -profile:v baseline -pix_fmt yuv420p '{final_video_path}' -y -hide_banner -loglevel error"

    if os.system(merge_cmd) != 0:
        print("❌ ترکیب نهایی شکست خورد."); return None

    return final_video_path

def successful():
    from google.colab import output
    output.eval_js('new Audio("https://huggingface.co/Toolsai/dubtest/resolve/main/ai.wav").play()')

In [ ]:

#@title 📤 آپلود ویدیو
import os
import re
import shutil
from google.colab import files
from IPython.display import clear_output

base_path = "/content"
upload_folder = os.path.join(base_path, "upload")
if not os.path.exists(upload_folder):
  os.makedirs(upload_folder)

for filename in os.listdir(upload_folder):
    os.remove(os.path.join(upload_folder, filename))

os.chdir(upload_folder)

print("لطفا فایل ویدئویی خود را برای بهبود کیفیت انتخاب کنید...")
uploaded = files.upload()

clear_output()

processing_file_path = ""

if uploaded:
  uploaded_filename = list(uploaded.keys())[0]
  _, file_extension = os.path.splitext(uploaded_filename)

  fixed_filename = f"input_video{file_extension}"
  processing_file_path = os.path.join(upload_folder, fixed_filename)

  os.rename(
      os.path.join(upload_folder, uploaded_filename),
      processing_file_path
  )

  print(f"✅ فایل با موفقیت آپلود و برای پردازش آماده شد:")
  print(processing_file_path)
else:
  print("❌ هیچ فایلی آپلود نشد.")

In [ ]:

#@title ▶️ شروع عملیات بهبود کیفیت
if 'processing_file_path' in locals() and os.path.exists(processing_file_path):
    save_path = run_gfpgan_and_create_final_video(processing_file_path)

    if save_path:
        successful()
        print(f"\n🎉 فرآیند با موفقیت به پایان رسید! فایل نهایی در مسیر زیر ذخیره شد:")
        print(save_path)
        print("\nدر حال نمایش پیش‌نمایش ویدیو:")
        display(Video(save_path, embed=True, width=400))
        print("\nدر حال شروع دانلود خودکار...")
        files.download(save_path)
    else:
        print("\n❌ پردازش با خطا مواجه شد.")
else:
  print("❌ لطفا ابتدا از سلول آپلود یک فایل ویدئویی آپلود کنید.")

به دلیل زمان‌بر بودن حین ‌ انجام پروسه میتونید به کارهای دیگه برسید. به محض تکمیل ، ویدیو بصورت خودکار دانلود میشه . محیط کولب نباید بسته شود و صفحه مانیتور ترجیحا روشن بماند